In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/indian-stock-index-1minute-data-2008-2020/BANK_NIFTY_data/BNF_2010.csv
/kaggle/input/indian-stock-index-1minute-data-2008-2020/BANK_NIFTY_data/BNF_2016.csv
/kaggle/input/indian-stock-index-1minute-data-2008-2020/BANK_NIFTY_data/BNF_2011.csv
/kaggle/input/indian-stock-index-1minute-data-2008-2020/BANK_NIFTY_data/BNF_2020.csv
/kaggle/input/indian-stock-index-1minute-data-2008-2020/BANK_NIFTY_data/BNF_2018.csv
/kaggle/input/indian-stock-index-1minute-data-2008-2020/BANK_NIFTY_data/BNF_2010_2020.csv
/kaggle/input/indian-stock-index-1minute-data-2008-2020/BANK_NIFTY_data/BNF_2014.csv
/kaggle/input/indian-stock-index-1minute-data-2008-2020/BANK_NIFTY_data/BNF_2019.csv
/kaggle/input/indian-stock-index-1minute-data-2008-2020/BANK_NIFTY_data/BNF_2017.csv
/kaggle/input/indian-stock-index-1minute-data-2008-2020/BANK_NIFTY_data/BNF_2015.csv
/kaggle/input/indian-stock-index-1minute-data-2008-2020/BANK_NIFTY_data/BNF_2012.csv
/kaggle/input/indian-stock-index-1minute-data-2008-2020/BANK

In [2]:
df19 = pd.read_csv('/kaggle/input/indian-stock-index-1minute-data-2008-2020/NIFTY_data/NIFTY_2019.csv',parse_dates=[['Date','Time']])
df19.set_index('Date_Time',inplace=True,drop=True)

In [3]:
!pip install backtesting
!pip install ta

     |████████████████████████████████| 175 kB 291 kB/s            
  Preparing metadata (setup.py) ... - done
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173823 sha256=9ab1eac98da4570f9c178a39ddfa154bf138ae48102376932aef00d4645e075f
  Stored in directory: /root/.cache/pip/wheels/48/1d/ea/65dacebc37da7655d8a1fd0f315ac39d102e31d6545237a1c6
Successfully built backtesting
  Preparing metadata (setup.py) ... - done
  Created wheel for ta: filename=ta-0.8.0-py3-none-any.whl size=28895 sha256=969a05fd538b957064523d5acf7b3958bf4a4dd6333744fc120e4c9882f5532f
  Stored in directory: /root/.cache/pip/wheels/7e/da/86/65cba22446ae2ef148de2079907264ef27feecfb7f51a45e0d
Successfully built ta


In [4]:
import backtesting
backtesting.set_bokeh_output(notebook=True)
from backtesting import Backtest,Strategy
from backtesting.lib import crossover
import ta.trend as tr

/opt/conda/lib/python3.7/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [5]:
class EmaSmaCross(Strategy):
    def init(self):
        cp = pd.Series(self.data.Close)
        
        self.ema1 = self.I(tr.ema_indicator,close=cp, window=2001);
        self.ema2 = self.I(tr.sma_indicator,close=cp, window=100)
        

    def next(self):
        # Buy or sell on ema -> sma cross over
        # Have a 200 pts stop loss for long trades and 130 points for short trades
        if crossover(self.ema1, self.ema2):
            self.buy(sl=self.data.High-200)
        elif crossover(self.ema2, self.ema1):
            self.sell(sl=self.data.Low+130)
            
bt = Backtest(df19, EmaSmaCross, commission=.002,cash=100000,
              exclusive_orders=True)
# Run the backtest
stats = bt.run()
 
# Print the performance metrics for this strategy 
print(stats)

# Plot the price movements, trades and other meta-data such as drawdown etc.
bt.plot(resample=True);

Start                     2019-01-01 09:08:00
End                       2019-12-31 16:39:00
Duration                    364 days 07:31:00
Exposure Time [%]                   48.781042
Equity Final [$]                   72055.6908
Equity Peak [$]                   100305.6967
Return [%]                         -27.944309
Buy & Hold Return [%]               11.824899
Return (Ann.) [%]                  -28.615871
Volatility (Ann.) [%]                5.494698
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -28.296738
Avg. Drawdown [%]                   -7.566646
Max. Drawdown Duration      350 days 07:03:00
Avg. Drawdown Duration       89 days 06:33:00
# Trades                                  108
Win Rate [%]                        46.296296
Best Trade [%]                        0.99478
Worst Trade [%]                     -3.281042
Avg. Trade [%]                    

/opt/conda/lib/python3.7/site-packages/backtesting/_plotting.py:122: UserWarning: Data contains too many candlesticks to plot; downsampling to '1H'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "


In [6]:
stats._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-9,2204,3173,10786.4838,10783.75,24.6042,0.000253,2019-01-08 14:29:00,2019-01-11 11:45:00,2 days 21:16:00
1,9,3173,3835,10805.3175,10819.65,128.9925,0.001326,2019-01-11 11:45:00,2019-01-15 10:11:00,3 days 22:26:00
2,-9,3835,5317,10798.0107,10949.15,-1360.2537,-0.013997,2019-01-15 10:11:00,2019-01-21 09:41:00,5 days 23:30:00
3,9,5854,5979,10901.6097,10903.40,16.1127,0.000164,2019-01-22 12:20:00,2019-01-22 14:25:00,0 days 02:05:00
4,-9,5979,6395,10881.5932,10866.70,134.0388,0.001369,2019-01-22 14:25:00,2019-01-23 15:03:00,1 days 00:38:00
...,...,...,...,...,...,...,...,...,...,...
103,5,90932,91468,12239.6805,12210.20,-147.4025,-0.002409,2019-12-26 09:48:00,2019-12-27 12:25:00,1 days 02:37:00
104,-5,91468,92131,12185.7796,12207.70,-109.6020,-0.001799,2019-12-27 12:25:00,2019-12-31 10:50:00,3 days 22:25:00
105,5,92131,92274,12232.1154,12224.90,-36.0770,-0.000590,2019-12-31 10:50:00,2019-12-31 13:13:00,0 days 02:23:00
106,-5,92274,92378,12200.4502,12194.00,32.2510,0.000529,2019-12-31 13:13:00,2019-12-31 14:57:00,0 days 01:44:00


In [7]:
stats._trades.sort_values('Duration')

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
40,-7,41962,41966,11861.4795,11881.65,-141.1935,-0.001701,2019-06-14 09:26:00,2019-06-14 09:30:00,0 days 00:04:00
62,-7,63665,63669,10877.8507,10899.85,-153.9951,-0.002022,2019-09-06 11:15:00,2019-09-06 11:19:00,0 days 00:04:00
63,7,63669,63679,10921.6497,10898.45,-162.3979,-0.002124,2019-09-06 11:19:00,2019-09-06 11:29:00,0 days 00:10:00
90,-6,80757,80769,11872.1082,11892.60,-122.9508,-0.001726,2019-11-18 10:46:00,2019-11-18 10:58:00,0 days 00:12:00
48,-7,49839,49859,11598.5065,11562.90,249.2455,0.003070,2019-07-12 14:31:00,2019-07-12 14:51:00,0 days 00:20:00
...,...,...,...,...,...,...,...,...,...,...
10,-8,14019,15084,10713.2805,10863.15,-1198.9560,-0.013989,2019-02-21 09:49:00,2019-02-25 14:58:00,4 days 05:09:00
14,-8,16172,16958,10820.9647,10971.25,-1202.2824,-0.013888,2019-02-28 14:12:00,2019-03-05 14:42:00,5 days 00:30:00
32,-7,30397,31377,11670.6120,11602.30,478.1840,0.005853,2019-04-30 12:00:00,2019-05-06 09:26:00,5 days 21:26:00
101,6,86084,87549,12037.0761,11954.30,-496.6566,-0.006877,2019-12-06 11:07:00,2019-12-12 10:16:00,5 days 23:09:00
